In [4]:
import sys

# only needed if running on Aarete terminal server
#sys.path.append('c:/program files/python38/lib/site-packages')

In [112]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import os.path
import csv
import time
import logging
import numpy as np
from multiprocessing import Pool, Process
import threading

In [6]:
hospitals = pd.read_excel(r'/Users/amanchopra/Documents/Work/Aarete/Provider Transparency Project/Hospitals.xlsx', sheet_name='Hospitals')
hospitals.info()
hospitals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7597 entries, 0 to 7596
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   X                                             7596 non-null   float64       
 1   Y                                             7596 non-null   float64       
 2   OBJECTID                                      7596 non-null   float64       
 3   ID                                            7596 non-null   float64       
 4   NAME                                          7597 non-null   object        
 5   ADDRESS                                       7597 non-null   object        
 6   CITY                                          7597 non-null   object        
 7   STATE                                         7597 non-null   object        
 8   ZIP                                           7597 non-null   int64 

X             Y  OBJECTID          ID  \
0 -1.331889e+07  4.346975e+06       1.0   5793230.0   
1 -1.322651e+07  4.049626e+06       2.0  53391362.0   
2 -1.315620e+07  4.031978e+06       3.0  11190023.0   
3 -1.317190e+07  4.041752e+06       4.0  17090028.0   
4 -1.313208e+07  4.037270e+06       5.0  23691706.0   

                                                NAME  \
0                    CENTRAL VALLEY GENERAL HOSPITAL   
1  LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...   
2                  EAST LOS ANGELES DOCTORS HOSPITAL   
3          SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD   
4                      KINDRED HOSPITAL BALDWIN PARK   

                     ADDRESS             CITY STATE    ZIP           ZIP4  \
0    1025 NORTH DOUTY STREET          HANFORD    CA  93230  NOT AVAILABLE   
1             150 VIA MERIDA  WESTLAKE VILAGE    CA  91362  NOT AVAILABLE   
2    4060 WHITTIER BOULEVARD      LOS ANGELES    CA  90023  NOT AVAILABLE   
3     6245 DE LONGPRE AVENUE        HOLLYWOOD    CA  90028  NOT AVAILABLE   
4  14148 FRANCISQUITO AVENUE     BALDWIN PARK    CA  91706  NOT AVAILABLE   

   ... Downloadable Data Type Price URL  ChargeMaster  \
0  ...               NaN  NaN       NaN           NaN   
1  ...               NaN  NaN       NaN           NaN   
2  ...               NaN  NaN       NaN           NaN   
3  ...               NaN  NaN       NaN           NaN   
4  ...               NaN  NaN       NaN           NaN   

  Hospital Standard Charges Average Charges billed per DRg  \
0                       NaN                            NaN   
1                       NaN                            NaN   
2                       NaN                            NaN   
3                       NaN                            NaN   
4                       NaN                            NaN   

  Hospital Price Transparency charge info  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   

   Hospital price transparency case information  COVID Test  \
0                                           NaN         NaN   
1                                           NaN         NaN   
2                                           NaN         NaN   
3                                           NaN         NaN   
4                                           NaN         NaN   

   Shoppable Services  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 44 columns]

In [7]:
ny_hospitals = hospitals[hospitals['STATE'] == 'NY'].reset_index(drop=True)
ny_hospitals.shape

(275, 44)

In [119]:
def get_TQ_prov_info(driver, wait_time):
    logger.debug('Finding information tabs for provider. Common options are "Provider" and "Disclaimer".')
    info_tabs = driver.find_element_by_xpath('//div[contains(@class, "more-information-tabs")]').find_element_by_tag_name('ul').find_elements_by_tag_name('li')
    ind = None
    for i in range(len(info_tabs)):
        if info_tabs[i].text == 'Provider':
            ind = i
            break
    logger.debug(f'Found index of "Provider tab at index {ind}."')

    info_tabs[i].click()
    time.sleep(wait_time)
    logger.debug('Fetching provider name...')
    name = driver.find_element_by_xpath('//div[contains(@class, "hospital-info-title")]').text
    logger.debug('Fetching provider address...')
    address = driver.find_element_by_xpath('//div[contains(@class, "hospital-detail-map")]').text.split('\n')[1]
    return [name] + [address]

def get_comparison_stats(driver):
    stats = {}
    logger.debug('Fetching comparison statistics...')
    price_comp_stats = driver.find_element_by_xpath('//div[contains(@class, "hospital-pricing-comparison")]').find_elements_by_tag_name('ul')
    for stat in price_comp_stats:
        stats[stat.find_element_by_tag_name('h3').text] = stat.find_element_by_tag_name('h4').text
    return stats
    
def get_cash_price(driver):
    logger.debug('Fetching cash price...')
    return driver.find_element_by_xpath('//div[contains(@class, "cost-estimation-calculator")]').find_element_by_id('cashPriceAmount').text
    
def get_insurance_price(driver, wait_time):
    prices = {}
    try:
        driver.find_element_by_xpath('//div[contains(@class, "payment-select-button")]').click() # click insurance price button
        time.sleep(wait_time)
    except Exception:
        logger.warning('Insurance prices not available! Expect missing value in insurance price field.')
        return None
    
    ins_plans = Select(driver.find_element_by_id('insurance-plan-selection')).options[1:-1]
    for plan in ins_plans:
        plan_name = plan.text
        if plan_name in prices:
            continue
            
        logger.debug(f'Fetching insurance price for {plan_name}...')
        plan.click() # select plan in dropdown
        time.sleep(wait_time / 2)
        try:
            prices[plan_name] = driver.find_element_by_xpath('//div[contains(@class, "estimation-calculator")]').find_element_by_xpath('//*[@id="app"]/div[1]/div[1]/div[2]/div/div[1]/ul/li[1]/span').text
        except Exception:
            logger.warning(f'Insurance price not available for {plan_name}!')
            
    return prices
    
def get_data_for_provider(driver, prov_name, prov_zip_code, page_load_wait=1, element_visible_wait=3, max_pages=float('inf'), max_services=float('inf'), output_csv_path=None, return_df=False):
    logger.info(f'Fetching data for provider: {prov_name} with zipcode: {prov_zip_code}')
    
    i = 1
    
    if return_df:
        data = []

    while True and i <= max_pages:   
        prov_name_url = '-'.join(prov_name.lower().split(' '))
        logger.debug(f'Opening TH URL for provider: {prov_name} (page {i}).')
        driver.get(f'https://turquoise.health/service_offerings?q=&service_name=&location={prov_zip_code}&provider_name={prov_name_url}&page={i}&distance=10')
        time.sleep(page_load_wait)
        
        num_services = driver.find_elements_by_xpath('//div[contains(@class, "service-info-cont")]')
        time.sleep(element_visible_wait)
        num_services = min(len(num_services), max_services)
        
        if num_services == 0:
            logger.warning(f'No information available for provider: {prov_name}! Moving to next provider...')
            return None
        
        for j in range(num_services):
            logger.debug(f'Scraping service # {j} on page {i}.')
            logger.debug('Scraping service information...')
            
            try:
                servs = driver.find_elements_by_xpath('//div[contains(@class, "service-info-cont")]')
                time.sleep(element_visible_wait)
                serv_info = servs[j].text.split('\n')
            except Exception:
                logger.warning(f'No service information available for provider: {prov_name}!')
                continue
            
            logger.debug('Scraping rate information...')   
            try:
                rate_btns = driver.find_elements_by_xpath('//a[contains(@class, "rate-button")]')
                time.sleep(element_visible_wait)
                rate_btns[j].click()
                time.sleep(page_load_wait)
            except Exception:
                logger.warning(f'No rates available for provider: {prov_name} for service: {serv_info[1]}! This provider will not be added to output table.')
                continue
            
            try:
                prov_info = get_TQ_prov_info(driver, page_load_wait)
            except Exception:
                logger.warning(f'Could not fetch provider info for provider: {prov_name} for service: {serv_info[1]}!')
                prov_info = [None]
                
                
            try:
                comp_stats = get_comparison_stats(driver)
            except Exception:
                logger.warning(f'Could not fetch comparison stats for provider: {prov_name} for service: {serv_info[1]}!')
                comp_stats = None
            
            try:
                cash_price = get_cash_price(driver)
            except Exception:
                logger.warning(f'Could not fetch cash price for provider: {prov_name} for service: {serv_info[1]}!')
                cash_price = None
            
            try:
                ins_price = get_insurance_price(driver, page_load_wait)
            except Exception:
                logger.warning(f'Could not fetch insurance price for provider: {prov_name} for service: {serv_info[1]}!')
                ins_price = None
            
            driver.back()
            
            row = [prov_name, prov_zip_code] + serv_info[:2] + [cash_price, ins_price, comp_stats] + prov_info
            logger.debug('Appening row to table...')
            if output_csv_path: 
                 with open(output_csv_path, 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow(row)
            if return_df: data.append(row)

            time.sleep(page_load_wait + 5)

        i += 1
        
    if return_df:
        prov_data = pd.DataFrame(data, columns=['Provider', 'Zip Code', 'CPT', 'Service', 'Cash Price', 'Insurance Price', 'Comparison Statistics', 'TQH Name', 'TQH Address'])
        return prov_data
    
def scrape_provs(driver_options, provs_search_df, log_file_name='webscraper_progress.log', page_load_wait=1.5, element_visible_wait=2, max_pages=float('inf'), max_services=float('inf'), output_csv_path=None, return_df=False):
    # webdriver options
    chrome_options = Options()
    for option in driver_options:
        chrome_options.add_argument(f'--{option}')
        
    # create driver
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    
    # logging config
    global logger
    logger = logging.getLogger('webscraper')
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler(log_file_name, mode='w')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(fh)
    
    # set up return variable
    if return_df:
        pricing_df = pd.DataFrame()
    
    if output_csv_path:
        with open(output_csv_path, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(['Provider', 'Zip Code', 'CPT', 'Service', 'Cash Price', 'Insurance Price', 'Comparison Statistics', 'TQH Name', 'TQH Address'])

    for ind, row in provs_search_df.iterrows():
        name = row['NAME']
        zip_c = row['ZIP']
        prov_data = get_data_for_provider(driver, name, zip_c, page_load_wait=page_load_wait, element_visible_wait=element_visible_wait, output_csv_path=output_csv_path)
        
        if return_df:
            pricing_df = pd.concat([pricing_df, prov_data]).reset_index(drop=True)
            
    driver.quit()

In [120]:
output_csv_path = r'/Users/amanchopra/Documents/Work/Aarete/Provider Transparency Project'
driver_options = ['incognito', 'window-size=1920,1080', 'disable-gpu', 'disable-extensions', "proxy-server='direct://'", 'proxy-bypass-list=*', 'start-maximized', 'no-sandbox']

threads = []

for i, hospital_sub_df in enumerate(np.array_split(ny_hospitals, 5)):
    p = threading.Thread(target=scrape_provs, args=[driver_options, hospital_sub_df], kwargs={'output_csv_path': f'{output_csv_path}/prov_{i+1}_data.csv', 'element_visible_wait': 0, 'page_load_wait': 2, 'log_file_name': f'webscraper_progress_{i + 1}.log'})
    p.start()
    threads.append(p)





====== WebDriver manager ======




====== WebDriver manager ======


====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Current google-chrome version is 93.0.4577
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Get LATEST driver version for 93.0.4577
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/amanchopra/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache
Driver [/Users/amanchopra/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache
Driver [/Users/amanchopra/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache
Driver [/Users/amanchopra/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache
Driver [/Users/